In [1]:
#import os
#import geopandas as gpd
import pandas as pd
import numpy as np
#import requests
#import json
#import zipfile
import datetime
import time
#from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import shape
import utils
import importlib
from sklearn.mixture import BayesianGaussianMixture
import nimfa
import scipy
from scipy import stats



In [3]:
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\hildlars\\OneDrive - Universitetet i Oslo\\NordicMathCovid\\GitProjects\\NordicMobilityData_WorkingRepo\\PythonCode\\GaussianMixture\\utils.py'>

In [2]:
FolderPath = '../Data/NorSwe/' # Path to raw data files.

Sensors = ["01777V885181", # OK
"77275V885276", # OK
#"51812V1203972", # Missing: a bit 2018 and early 2019-late 2020. Wount capture pandemic change with this
"35829V885266", # Missing: 2017-2018
#"08581V885541", # Missing: 2017-mid 2019. Too little data for model fit, but include later? Not a lot of data, skip this one.
#"98823V578220", # Missing: 2017-mid 2019 and early 2020 and a bit 2022    Røyrvik, litt tvilsom
"99923V578123", # OK
#"93561V578187", # Missing: 2017-mid 2019 and a bit late 2019. Cannot compare before and after pandemic
"50089V578151", # Missing: 2017-late 2018 (+ start of pandemic?). ?????????
"84237V578097", # OK
#"11051V704737", # Missing: partly 2017 and 2021 - end. Drevsjø øst, litt tvilsom. Ikke post-pandemic data
"69140V704643", # OK
#"14158V705081", # Missing: 2017-2019 + sporadisk. Flermoberget, litt tvilsom ??????????
"00737V704646", # OK
"94864V704707", # Missing: sporadisk gjennom pandemien ????????
"94299V704696", # OK
"57929V705247", # Missing: A bit 2018/1019     Øyermoen, litt tvilsom
"76778V704564", # OK  Morokulien, litt tvilsom
"05732V971567", # Missing: 2017-mid 2017
"21405V2607269", # Missing: 2017-early 2019 NB!!!!!!!!
"09269V971425", # Missing: 2017-mid/late 2017
"52209V971422", # Missing 2017-late 2017    Prestbakke, litt tvilsom
"02535V971411", # Missing: 2017-late 2018
#"57474V971423", # Missing: 2017-late 2018 + sporadisk rundt aarsskiftene.   Berby, bittelitt tvilsom, men kanskje ikke
"04904V971774", # OK
"35229V971507"] # OK



In [3]:
fit_dates = {"01777V885181": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)],
"77275V885276": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # OK
#"51812V1203972":[datetime.date(2017, 1, 1),datetime.date(2018,12,31)], # Missing: a bit 2018 and early 2019-late 2020. No good data to fit
"35829V885266": [datetime.date(2018, 1, 15),datetime.date(2020,3,12)], # Missing: 2017-2018
"99923V578123": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # OK
#"98823V578220": [datetime.date(2019, 7, 1),datetime.date(2020,1,1)], # Missing: 2017-mid 2019 and early 2020 and a bit 2022.    Røyrvik, litt tvilsom. Not much data pre-pandemic
#"93561V578187": [datetime.date(2019, 7, 1),datetime.date(2020,3,12)], # Missing: 2017-mid 2019 and a bit late 2019. No good data to fit
"50089V578151": [datetime.date(2019, 1, 1),datetime.date(2019,12,31)], # Missing: 2017-late 2018 (+ start of pandemic?) ??????????????????
"84237V578097": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # OK
"76778V704564": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # OK  Morokulien, litt tvilsom
"69140V704643": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # OK
"57929V705247": [datetime.date(2017, 1, 1),datetime.date(2018,5,30)], # Missing: A bit 2018/1019     Øyermoen, litt tvilsom
"52209V971422": [datetime.date(2018, 1, 1),datetime.date(2020,3,12)], # Missing 2017-late 2017    Prestbakke, litt tvilsom
#"14158V705081": [datetime.date(2019, 2, 1),datetime.date(2020,3,12)], # Missing: 2017-2019 + sporadisk. Flermoberget, litt tvilsom ??????????
#"08581V885541": [datetime.date(2019, 9, 1),datetime.date(2020,3,12)], # # Missing: 2017-mid 2019. Too little data for model fit? Ikke pre-pandemic data
#"11051V704737": [datetime.date(2017, 8, 1),datetime.date(2020,3,12)], # Missing: partly 2017 and 2021 - end. Drevsjø øst, litt tvilsom. Ikke post-pandemic data
"00737V704646": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # OK
"94864V704707": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # Missing: sporadisk gjennom pandemien?????
"94299V704696": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # OK
"05732V971567": [datetime.date(2017, 10, 1),datetime.date(2020,3,12)], # Missing: 2017-mid 2017
"21405V2607269": [datetime.date(2019, 3, 1),datetime.date(2020,3,12)], # Missing: 2017-early 2019 NB!!!!!!!!
"09269V971425": [datetime.date(2018, 3, 1),datetime.date(2020,3,12)], # Missing: 2017-mid/late 2017
"02535V971411": [datetime.date(2019, 1, 1),datetime.date(2020,3,12)], # Missing: 2017-late 2018
#"57474V971423": [datetime.date(2018, 11, 1),datetime.date(2019,9,30)], # Missing: 2017-late 2018 + sporadisk rundt aarsskiftene.   Berby, bittelitt tvilsom, men kanskje ikke
"04904V971774": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)], # OK
"35229V971507": [datetime.date(2017, 1, 1),datetime.date(2020,3,12)] } # OK

Read data for each sensor, from .csv files downloaded using the function 'get_traffic_NOR()' in the file road_traffic_data_methodologyPaper.ipynb

In [4]:

sensor_dict = {}
all_data = []
for s in Sensors:
    sensor_data = pd.read_csv(FolderPath+s+'_by_length_hour.csv', sep=',',usecols= ['sensor_id', 'from_date', 'to_date', 'from_hour','to_hour', 'sensor_dir', 'short_vehicles', 'long_vehicles', 'unknown_length'], parse_dates=['from_date','to_date'])
    min_date = sensor_data.from_date.min()
    max_date = sensor_data.to_date.max()
    sensor_directions = sensor_data.sensor_dir.unique()
    sensor_dict[s] = {'Start' : min_date, 'End' : max_date, 'Directions' : sensor_directions}

    all_data.append(sensor_data)





Change sensor direction to NOR and SWE

In [5]:
to_norway = ['Trældal x Ev6', 'Hestbrinken', 'Mo i Rana','Trofors','Hattfjelldalen', 'Gjersvika','Sandvika','Nordli','Verdalsøra','Meråker','Drevsjø','ØSTBY','X/RV 25','NYBERGSUND','Holtet','Røgden','ØYERMOEN XF202','KONGSVINGER','BEKKENGA','Oslo','Halden','HALDEN','OSLO']

for df in all_data:
    df['sensor_dir'] = np.where(list(i in to_norway for i in df['sensor_dir']), 'NOR','SWE')
    


In [6]:
def agg_APIdata_NORSWE(data,WhichVehicles,min_date = datetime.datetime(2017, 1, 1)  , max_date = datetime.datetime(2023, 5, 22)):

#WhichVehicles can be 'Small', 'Heavy', 'Total' or 'Both' 

    # What vehicle lengths do we want
    if WhichVehicles == 'Total':
        data['total_vehicles'] = data['short_vehicles'] + data['long_vehicles'] + data['unknown_length']
    elif WhichVehicles == 'Small':
        data = data.rename(columns={'short_vehicles':'total_vehicles'})
    elif WhichVehicles == 'Heavy':
        data = data.rename(columns={'long_vehicles': 'total_vehicles'})

    # Create sensor direction and origin columns
    data = data.rename(columns = {'sensor_dir': 'dest_country'})
    data['origin_country'] = np.where(data['dest_country'] == 'NOR', 'SWE','NOR')

    data['sensor_id'] = data['sensor_id'].astype(str)

    data['sensor_origin'] = data[['sensor_id', 'origin_country']].agg(', '.join, axis=1)
    data['sensor_destination'] = data[['sensor_id', 'dest_country']].agg(', '.join, axis=1)

    ## Add some temporal informaiton. 

    data = data.drop(['to_date'], axis = 1).rename(columns = {'from_date':'date'})

    data['minute'] = datetime.timedelta(minutes = 0)


    data = data[~(data['from_hour'] == data['to_hour'])].rename(columns = {'from_hour':'hour'})

    data['hour'] = pd.to_timedelta(data['hour'].apply(lambda x: int(x[:2])), unit='h')

    data['date'] = data['date'] + data['hour'] + data['minute']
    data = data[(data.date > min_date) & (data.date < max_date)].copy()

    if WhichVehicles == 'Both':
        data = data[['sensor_origin','sensor_destination','date','small_vehicles','long_vehicles','unknown_length']].copy()

    else:
        data = data[['sensor_origin', 'sensor_destination', 'date', 'total_vehicles']].copy().reset_index()
        rm_idx = np.where(np.isnan(data.total_vehicles))
        data = data.drop(index = rm_idx[0])
        data['total_vehicles'] = data['total_vehicles'].apply(lambda x: int(x))

        f = lambda x: x.reindex(pd.date_range(min_date,
                                                max_date,
                                                name='date',
                                                freq='1h'), fill_value=0)


        data = (data.set_index('date')
                    .groupby(["sensor_origin", "sensor_destination"])["total_vehicles"]
                    .apply(f)
                    .reset_index())
        data = data.pivot_table(index=["sensor_origin","sensor_destination"], columns=["date"],values=["total_vehicles"] ).droplevel(level = 0,axis = 1 )

    return(data)

In [7]:
## For 2017-2023 this takes about 2 min to run
agg_data = []
for df in all_data:
    d = agg_APIdata_NORSWE(df, 'Small',min_date = datetime.datetime(2017, 1, 1)  , max_date = datetime.datetime(2023, 12, 31))
    agg_data.append(d)
    

In [8]:
models = []
data = []

for df in agg_data:
    sens = df.index[0][0].split(',')[0]
    d1 = fit_dates[sens][0]
    d2 = fit_dates[sens][1]
    mod, dat = utils.fit_period(df, d1=d1, d2=d2, hourly = True, nSamp = 10000, Normalize = False, N = 10, seed=2024, FitMethod = 'Bayesian')
    models.append(mod)
    data.append(dat) 

models = pd.concat(models,axis=1)
data = pd.concat(data,axis=0)
agg_data = pd.concat(agg_data,axis=0)

# Save the fitted model and data.
models.to_pickle('../Data/models_nor.pkl')
data.to_pickle('../Data/data_nor.pkl')
agg_data.to_pickle('../Data/agg_data_nor.pkl')

c:\Users\hildlars\OneDrive - Universitetet i Oslo\NordicMathCovid\PanNordicMobility\FinalFiles_commutingPaper\Code\utils.py:122: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  models = data[c].reset_index("weekday").groupby("weekday").agg(lambda x : fit_model(x,hourly_data, nSamp, N = N, hourminute = True,seed = seed, FitMethod = FitMethod))
c:\Users\hildlars\OneDrive - Universitetet i Oslo\NordicMathCovid\PanNordicMobility\FinalFiles_commutingPaper\Code\utils.py:122: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  models = data[c].reset_index("weekday").groupby("weekday").agg(lambda x : fit_model(x,hourly_data, nSamp, N = N, hourminute = True,seed = seed, FitMethod = FitMethod))
c:\Users\hildlars\OneDrive - Universitetet i Oslo\NordicMathCovid\PanNordicMobility\FinalFiles_commutingPaper\Code\utils.py:122: PerformanceWarning: dropping on a non-lexsorted multi